## Banning Transit is missing
[PR](https://github.com/cal-itp/data-analyses/issues/1254)

In [1]:
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

In [2]:
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
import _operators_prep as op_prep

In [5]:
analysis_date_list = rt_dates.y2024_dates + rt_dates.y2023_dates

## 11/26: why isn't `deploy_portfolio_yaml` working

In [6]:
df = (
    op_prep.operators_schd_vp_rt()
    .sort_values(["caltrans_district", "organization_name"])
    .reset_index(drop=True)
)

In [7]:
df.head()

,caltrans_district,organization_name
0,01 - Eureka,Blue Lake Rancheria
1,01 - Eureka,City of Arcata
2,01 - Eureka,City of Eureka
3,01 - Eureka,Curry Public Transit
4,01 - Eureka,Humboldt Transit Authority


In [8]:
df.caltrans_district.value_counts()

07 - Los Angeles        55
04 - Oakland            35
03 - Marysville         18
10 - Stockton           16
05 - San Luis Obispo    12
06 - Fresno             12
08 - San Bernardino     11
01 - Eureka              9
02 - Redding             9
11 - San Diego           6
12 - Irvine              3
09 - Bishop              1
Name: caltrans_district, dtype: int64

In [9]:
import yaml
from pathlib import Path

In [10]:
portfolio_site_yaml = Path("../portfolio/sites/gtfs_digest.yml")

In [11]:
chapter_info = {
        "column": "caltrans_district",
        "name": "district",
        "caption_prefix": "District ",
        "caption_suffix": "",
    }

In [12]:
section_info = {
        "column": "organization_name",
        "name": "organization_name",
    }

In [13]:
chapter_col = chapter_info["column"]
chapter_values = sorted(list(df[chapter_col].unique()))

In [14]:
with open(portfolio_site_yaml) as f:
    site_yaml_dict = yaml.load(f, yaml.Loader)

In [15]:
section_col = section_info["column"]
caption_prefix = chapter_info["caption_prefix"]
caption_suffix = chapter_info["caption_suffix"]

In [16]:
chapter_info["name"]

'district'

In [17]:
caption_prefix

'District '

In [18]:
chapters_list = [
        {
            **{
                "caption": f"{caption_prefix}{one_chapter_value}{caption_suffix}",
                "params": {chapter_info["name"]: str(one_chapter_value)},
                "sections": [
                    {section_info["name"]: str(one_section_value)}
                    for one_section_value in df[df[chapter_col] == one_chapter_value][section_col].unique().tolist()
                ],
            }
        }
        for one_chapter_value in chapter_values
    ]

In [19]:
chapters_list

[{'caption': 'District 01 - Eureka',
  'params': {'district': '01 - Eureka'},
  'sections': [{'organization_name': 'Blue Lake Rancheria'},
   {'organization_name': 'City of Arcata'},
   {'organization_name': 'City of Eureka'},
   {'organization_name': 'Curry Public Transit'},
   {'organization_name': 'Humboldt Transit Authority'},
   {'organization_name': 'Lake Transit Authority'},
   {'organization_name': 'Mendocino Transit Authority'},
   {'organization_name': 'POINT'},
   {'organization_name': 'Redwood Coast Transit Authority'}]},
 {'caption': 'District 02 - Redding',
  'params': {'district': '02 - Redding'},
  'sections': [{'organization_name': 'Lassen Transit Service Agency'},
   {'organization_name': 'Modoc Transportation Agency'},
   {'organization_name': 'Plumas Transit Systems'},
   {'organization_name': 'Redding Area Bus Authority'},
   {'organization_name': 'Shasta County'},
   {'organization_name': 'Siskiyou County'},
   {'organization_name': 'Susanville Indian Rancheria'},

In [20]:
parts_list = [{"chapters": chapters_list}]

In [21]:
parts_list

[{'chapters': [{'caption': 'District 01 - Eureka',
    'params': {'district': '01 - Eureka'},
    'sections': [{'organization_name': 'Blue Lake Rancheria'},
     {'organization_name': 'City of Arcata'},
     {'organization_name': 'City of Eureka'},
     {'organization_name': 'Curry Public Transit'},
     {'organization_name': 'Humboldt Transit Authority'},
     {'organization_name': 'Lake Transit Authority'},
     {'organization_name': 'Mendocino Transit Authority'},
     {'organization_name': 'POINT'},
     {'organization_name': 'Redwood Coast Transit Authority'}]},
   {'caption': 'District 02 - Redding',
    'params': {'district': '02 - Redding'},
    'sections': [{'organization_name': 'Lassen Transit Service Agency'},
     {'organization_name': 'Modoc Transportation Agency'},
     {'organization_name': 'Plumas Transit Systems'},
     {'organization_name': 'Redding Area Bus Authority'},
     {'organization_name': 'Shasta County'},
     {'organization_name': 'Siskiyou County'},
     {

In [22]:
site_yaml_dict["parts"] = parts_list

In [23]:
site_yaml_dict

{'directory': './gtfs_digest/',
 'notebook': './gtfs_digest/03_report.ipynb',
 'parts': [{'chapters': [{'caption': 'District 01 - Eureka',
     'params': {'district': '01 - Eureka'},
     'sections': [{'organization_name': 'Blue Lake Rancheria'},
      {'organization_name': 'City of Arcata'},
      {'organization_name': 'City of Eureka'},
      {'organization_name': 'Curry Public Transit'},
      {'organization_name': 'Humboldt Transit Authority'},
      {'organization_name': 'Lake Transit Authority'},
      {'organization_name': 'Mendocino Transit Authority'},
      {'organization_name': 'POINT'},
      {'organization_name': 'Redwood Coast Transit Authority'}]},
    {'caption': 'District 02 - Redding',
     'params': {'district': '02 - Redding'},
     'sections': [{'organization_name': 'Lassen Transit Service Agency'},
      {'organization_name': 'Modoc Transportation Agency'},
      {'organization_name': 'Plumas Transit Systems'},
      {'organization_name': 'Redding Area Bus Authori

In [24]:
output = yaml.dump(site_yaml_dict)

In [26]:
with open(portfolio_site_yaml, "w") as f:
        f.write(output)